# Get data from the python client

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Swagger UI)[https://control.green-fusion.de/services/energy-management-backend/v1/api/doc#/]


## Inits

In [1]:
#imports
from energy_management_client import BackendPythonClient
import pandas as pd
import json

In [2]:
# select your environment
environment="dev" # prod or dev

# Selector
if environment=="dev":
    backend_endpoint_path="https://dev.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="development"
elif environment=="prod":
    backend_endpoint_path="https://control.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="control"  
else:
    backend_endpoint_path=""
    realm_type=""
    print("select either prod or dev for the environment")

#test
#backend_endpoint_path, realm_type
if True:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint=backend_endpoint_path,)
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm=realm_type,
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

In [3]:
if False:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint="https://control.green-fusion.de/services/energy-management-backend/v1/api", )
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm="control",
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

# Functions

In [4]:
def get_buildings_df(client, customers):
    """
    Loop through all customers, fetch their buildings, and build a DataFrame where each row contains:
      - building_id, customerID, address, city, atvise_display_name, base,
      - heatingType, heatingSurface, access_key, nickname, postal_code, gui_name,
      - coordinates, CustomerAlias gfid,
      - LayoutID (extracted from modular_system)
    """
    try:
        data = []
        for c in customers:
            # Configure the client for this customer
            customer_id = c.id
            client.configure(customer_id=customer_id)
            
            # Get the buildings for this customer (returns a BuildingsResponseDto)
            c_buildings = client.buildings.get_buildings()
            building_list = getattr(c_buildings, 'buildings', [])
            
            for b in building_list:
                # Extract building fields (using attribute access)
                building_id        = getattr(b, 'building_id', None)
                customerID         = getattr(b, 'customerID', None)
                address            = getattr(b, 'address', None)
                city               = getattr(b, 'city', None)
                atvise_display_name= getattr(b, 'atvise_display_name', None)
                base               = getattr(b, 'base', None)
                heatingType        = getattr(b, 'heatingType', None)
                heatingSurface     = getattr(b, 'heatingSurface', None)
                access_key         = getattr(b, 'access_key', None)
                nickname           = getattr(b, 'nickname', None)
                postal_code        = getattr(b, 'postal_code', None)
                gui_name           = getattr(b, 'gui_name', None)
                coordinates        = getattr(b, 'coordinates', None)
                CustomerAlias      = getattr(b, 'CustomerAlias', None)
                gfid               = getattr(b, 'gfid', None)
                
                # Parse the modular_system JSON and extract layoutID
                modular_system_str = getattr(b, 'modular_system', None)
                LayoutID = None
                if modular_system_str:
                    try:
                        modular_system = json.loads(modular_system_str)
                        LayoutID = modular_system.get('layoutID', None)
                    except json.JSONDecodeError:
                        LayoutID = None

                data.append({
                    "building_id": building_id,
                    "customerID": customerID,
                    "address": address,
                    "city": city,
                    "atvise_display_name": atvise_display_name,
                    "base": base,
                    "heatingType": heatingType,
                    "heatingSurface": heatingSurface,
                    "access_key": access_key,
                    "nickname": nickname,
                    "postal_code": postal_code,
                    "gui_name": gui_name,
                    "coordinates": coordinates,
                    "CustomerAlias": CustomerAlias,
                    "gfid": gfid,
                    "LayoutID": LayoutID
                })

        return pd.DataFrame(data)
    
    except Exception as e:
        print("API Call Failed:", e)
        return pd.DataFrame()

# Usage:
# customers = client.customers.get_list_of_customers()
# df = get_buildings_df(client, customers)
# print(df.head())

# usage get_buildings_df

In [5]:
# usage get_buildings_df
if True:
    # Get customers list (list of dicts)
    customers = client.customers.get_list_of_customers()
    #build df
    df = get_buildings_df(client, customers)
    #test
    df.head()

In [6]:
df[['building_id', 'customerID','gfid','LayoutID']].head()

,building_id,customerID,gfid,LayoutID
0,571,None,GFP-7066,gas:1--gas:2--global-separation-circuit:1--hea...
1,574,None,GFP-7141,gas:1--gas:2--global-separation-circuit:1--hea...
2,575,None,GFP-7166,gas:1--gas:2--global-separation-circuit:1--hea...
3,738,None,GFP-12968,gas:1--gas:2--chp:1--buffer-tank:1--global-sep...
4,739,None,GFP-12991,gas:1--gas:2--chp:1--buffer-tank:1--global-sep...


# Exports

In [7]:
df.columns

Index(['building_id', 'customerID', 'address', 'city', 'atvise_display_name',
       'base', 'heatingType', 'heatingSurface', 'access_key', 'nickname',
       'postal_code', 'gui_name', 'coordinates', 'CustomerAlias', 'gfid',
       'LayoutID'],
      dtype='object')

In [8]:
# get_buildings_df
if True:
    df[['building_id','customerID', 'address', 'postal_code', 'city', 'atvise_display_name',
       'base', 'heatingType', 'heatingSurface', 'nickname', 'gui_name', 'coordinates', 'CustomerAlias', 'gfid',
       'LayoutID']].to_csv('data_from_db.csv', index=False)